In [1]:
!nvidia-smi

Tue Nov 14 00:47:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


In [3]:
%matplotlib inline
!git clone https://github.com/VikashKumarShaw1994/DNN_invariant_time-rescaling.git DNN_invariant_sithcon

fatal: destination path 'DNN_invariant_sithcon' already exists and is not an empty directory.


In [3]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
from DNN_invariant_sithcon.SITHCon.sithcon import SITHCon_Layer
#from sithcon import SITHCon_Layer, _SITHCon_Core, iSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os
from os.path import join
import glob

import numpy as np
import pandas as pd
import pickle
from math import factorial
import random

<class 'torch.cuda.FloatTensor'>


In [4]:
Braille_DICT = {
    ',': [
        [' ', '●'],
        [' ', ' '],
        [' ', ' '],
        [' ', ' ']
    ],
    '.': [
        [' ', '●'],
        [' ', ' '],
        ['●', '●'],
        [' ', ' ']
    ],
    '(': [
        [' ', '●'],
        [' ', ' '],
        ['●', ' '],
        [' ', ' ']
    ],
    '-': [
        [' ', '●'],
        ['●', ' '],
        [' ', ' '],
        [' ', ' ']
    ],
    '/': [
        [' ', ' '],
        ['●', ' '],
        [' ', ' '],
        [' ', ' ']
    ],
    '?': [
        [' ', '●'],
        ['●', ' '],
        [' ', '●'],
        [' ', ' ']
    ],
    ')': [
        [' ', '●'],
        ['●', ' '],
        ['●', ' '],
        [' ', ' ']
    ],
    'A': [
        ['●', ' '],
        [' ', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'B': [
        ['●', '●'],
        [' ', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'C': [
        ['●', ' '],
        [' ', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'D': [
        ['●', ' '],
        [' ', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'E': [
        ['●', ' '],
        [' ', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'F': [
        ['●', '●'],
        [' ', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'G': [
        ['●', '●'],
        [' ', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'H': [
        ['●', '●'],
        [' ', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'I': [
        [' ', '●'],
        [' ', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'J': [
        [' ', '●'],
        [' ', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'K': [
        ['●', ' '],
        ['●', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'L': [
        ['●', '●'],
        ['●', ' '],
        [' ', ' '],
        ['●', ' ']
    ],
    'M': [
        ['●', ' '],
        ['●', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'N': [
        ['●', ' '],
        ['●', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'O': [
        ['●', ' '],
        ['●', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'P': [
        ['●', '●'],
        ['●', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'Q': [
        ['●', '●'],
        ['●', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'R': [
        ['●', '●'],
        ['●', ' '],
        ['●', ' '],
        ['●', ' ']
    ],
    'S': [
        [' ', '●'],
        ['●', '●'],
        [' ', ' '],
        ['●', ' ']
    ],
    'T': [
        [' ', '●'],
        ['●', '●'],
        ['●', ' '],
        ['●', ' ']
    ],
    'U': [
        ['●', ' '],
        ['●', ' '],
        [' ', '●'],
        ['●', ' ']
    ],
    'V': [
        ['●', '●'],
        ['●', ' '],
        [' ', '●'],
        ['●', ' ']
    ],
    'W': [
        [' ', '●'],
        [' ', '●'],
        ['●', '●'],
        ['●', ' ']
    ],
    'X': [
        ['●', ' '],
        ['●', '●'],
        [' ', '●'],
        ['●', ' ']
    ],
    'Y': [
        ['●', ' '],
        ['●', '●'],
        ['●', '●'],
        ['●', ' ']
    ],
    'Z': [
        ['●', ' '],
        ['●', ' '],
        ['●', '●'],
        ['●', ' ']
    ],
    '1': [
        ['●', ' '],
        [' ', ' '],
        [' ', ' '],
        [' ', '●']
    ],
    '2': [
        ['●', '●'],
        [' ', ' '],
        [' ', ' '],
        [' ', '●']
    ],
    '3': [
        ['●', ' '],
        [' ', '●'],
        [' ', ' '],
        [' ', '●']
    ],
    '4': [
        ['●', ' '],
        [' ', '●'],
        ['●', ' '],
        [' ', '●']
    ],
    '5': [
        ['●', ' '],
        [' ', ' '],
        ['●', ' '],
        [' ', '●']
    ],
    '6': [
        ['●', '●'],
        [' ', '●'],
        [' ', ' '],
        [' ', '●']
    ],
    '7': [
        ['●', '●'],
        [' ', '●'],
        ['●', ' '],
        [' ', '●']
    ],
    '8': [
        ['●', '●'],
        [' ', ' '],
        ['●', ' '],
        [' ', '●']
    ],
    '9': [
        [' ', '●'],
        [' ', '●'],
        [' ', ' '],
        [' ', '●']
    ],
    '0': [
        [' ', '●'],
        [' ', '●'],
        ['●', ' '],
        [' ', '●']
    ]
}


In [6]:
def braille_to_binary(braille_matrix):
    braille_code = ""
    for row in range(4):
        for col in range(2):
            braille_code += '1' if braille_matrix[row][col] == '●' else '0'

    return braille_code

Braille_CODE_DICT={}
for brk,brv in Braille_DICT.items():
    Braille_CODE_DICT[brk]=braille_to_binary(brv)
print(Braille_CODE_DICT)

{',': '01000000', '.': '01001100', '(': '01001000', '-': '01100000', '/': '00100000', '?': '01100100', ')': '01101000', 'A': '10000010', 'B': '11000010', 'C': '10010010', 'D': '10011010', 'E': '10001010', 'F': '11010010', 'G': '11011010', 'H': '11001010', 'I': '01010010', 'J': '01011010', 'K': '10100010', 'L': '11100010', 'M': '10110010', 'N': '10111010', 'O': '10101010', 'P': '11110010', 'Q': '11111010', 'R': '11101010', 'S': '01110010', 'T': '01111010', 'U': '10100110', 'V': '11100110', 'W': '01011110', 'X': '10110110', 'Y': '10111110', 'Z': '10101110', '1': '10000001', '2': '11000001', '3': '10010001', '4': '10011001', '5': '10001001', '6': '11010001', '7': '11011001', '8': '11001001', '9': '01010001', '0': '01011001'}


In [7]:
Braille_code_numpy = {key:np.array([int(x) for x in Braille_CODE_DICT[key]] + [0, 0])
                    for key in Braille_CODE_DICT.keys()}

subset = list(Braille_code_numpy.keys())

id2key = subset
key2id = {}
for idx, s in enumerate(subset):
    key2id[s] = idx

X = [ttype(Braille_code_numpy[k])for k in subset]
Y = torch.LongTensor(np.arange(0,len(X)))
print(X, Y)

[tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 0., 0., 1., 1., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 1., 0., 0., 1., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 1., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 1., 1., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 0., 1., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 1., 0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 1., 1., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 1., 0., 1., 0., 0., 0.], device='cuda

<ipython-input-7-a3c04df30030>:11: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  X = [ttype(Braille_code_numpy[k])for k in subset]


In [8]:
class SITHCon_Classifier(nn.Module):
    def __init__(self, out_classes, layer_params,
                 act_func=nn.ReLU, batch_norm=False,
                 dropout=.2):
        super(SITHCon_Classifier, self).__init__()
        last_channels = layer_params[-1]['channels']
        self.transform_linears = nn.ModuleList([nn.Linear(l['channels'], l['channels'])
                                                for l in layer_params])
        self.sithcon_layers = nn.ModuleList([SITHCon_Layer(l, act_func) for l in layer_params])
        self.to_out = nn.Linear(last_channels, out_classes)


    def forward(self, inp):

        x = inp
        #out = []
        for i in range(len(self.sithcon_layers)):
            x = self.sithcon_layers[i](x)

            x = self.transform_linears[i](x[:,0,:,:].transpose(1,2))
            x = x.unsqueeze(1).transpose(2,3)

            #out.append(x.clone())
        x = x.transpose(2,3)[:, 0, :, :]
        #x = x.transpose(2,3)[:, 0, :, :]
        x = self.to_out(x)
        return x

# Three Layers

In [9]:
p = [400, 35, 23, 2]

sp1 = dict(in_features=1,
           tau_min=.1, tau_max=4000, buff_max=6500,
           dt=1, ntau=p[0], k=p[1], g=0.0, ttype=ttype,
           channels=35, kernel_width=p[2], dilation=p[3],
           dropout=None, batch_norm=None)
sp2 = dict(in_features=sp1['channels'],
           tau_min=.1, tau_max=4000, buff_max=6500,
           dt=1, ntau=p[0], k=p[1], g=0.0, ttype=ttype,
           channels=35, kernel_width=p[2], dilation=p[3],
           dropout=None, batch_norm=None)
sp3 = dict(in_features=sp2['channels'],
           tau_min=.1, tau_max=4000, buff_max=6500,
           dt=1, ntau=p[0], k=p[1], g=0.0, ttype=ttype,
           channels=35, kernel_width=p[2], dilation=p[3],
           dropout=None, batch_norm=None)

# TWO LAYERS
layer_params = [sp1, sp2]#, sp3]


model = SITHCon_Classifier(len(X), layer_params, act_func=nn.ReLU).cuda()
model
tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)

{'tau_min': 0.1, 'tau_max': 4000, 'buff_max': 6500, 'dt': 1, 'ntau': 400, 'k': 35, 'g': 0.0, 'ttype': <class 'torch.cuda.FloatTensor'>, 'dropout': None, 'batch_norm': None}


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


{'tau_min': 0.1, 'tau_max': 4000, 'buff_max': 6500, 'dt': 1, 'ntau': 400, 'k': 35, 'g': 0.0, 'ttype': <class 'torch.cuda.FloatTensor'>, 'dropout': None, 'batch_norm': None}
Total Weights: 33118
SITHCon_Classifier(
  (transform_linears): ModuleList(
    (0-1): 2 x Linear(in_features=35, out_features=35, bias=True)
  )
  (sithcon_layers): ModuleList(
    (0): SITHCon_Layer(
      (tctct): _SITHCon_Core(
        (sith): iSITH(ntau=400, tau_min=0.1, tau_max=4000, buff_max=6500, dt=1, k=35, g=0.0)
        (conv): Conv2d(1, 35, kernel_size=(1, 23), stride=(1, 1), dilation=(1, 2), bias=False)
        (maxp): MaxPool1d(kernel_size=356, stride=356, padding=0, dilation=1, ceil_mode=False)
      )
      (act_func): ReLU()
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (1): SITHCon_Layer(
      (tctct): _SITHCon_Core(
        (sith): iSITH(ntau=400, tau_min=0.1, tau_max=4000, buff_max=6500, dt=1, k=35, g=0.0)
        (conv): Conv2d(1, 35, kernel_size=(35, 23), stride=(1, 1), dilation=(1,

In [10]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 5000
Trainscale = 10
device='cuda'
batch_size = 8
batches = int(np.ceil(43 / batch_size))
progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
times_100 = 0
for epoch_idx in progress_bar:
    perfs = []
    losses = []
    model.train()
    for batch_idx in range(batches):
        optimizer.zero_grad()
        loss = 0
        permute = np.arange(0, 43)
        for i in range(0, int(min(len(X) - (batch_idx*batch_size),
                              batch_size))
                       ):
            iv = X[permute[batch_idx*batch_size + i]]
            iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(0).to(device)
            iv = iv.unsqueeze(-1)
            iv = iv.repeat(1,1,1,1,Trainscale)
            iv = iv.reshape(1,1,1,-1)
            tv = Y[permute[batch_idx*batch_size + i]].to(device)
            optimizer.zero_grad()
            out = model(iv)
            loss += loss_func(out[:, -1, :],
                             torch.cuda.LongTensor([tv]))
            perfs.append((torch.argmax(out[:, -1, :], dim=-1) ==
                      tv).sum().item())
        loss = loss / min(len(X) - (batch_idx*batch_size),
                          batch_size)
        loss.backward()
        optimizer.step()


        #perfs = perfs[int(-loss_buffer_size/batch_size):]
        losses.append(loss.detach().cpu().numpy())
        #losses = losses[int(-loss_buffer_size/batch_size):]


        s = "{}:{:2} Loss: {:.4f}, Perf: {:.4f}"
        format_list = [epoch_idx, batch_idx, np.mean(losses),
                       np.sum(perfs)/((len(perfs)))]
        s = s.format(*format_list)
        progress_bar.set_description(s)
    if (np.sum(perfs)/((len(perfs))) == 1.0) & (np.mean(losses) < .11):
        times_100 += 1
        if times_100 >= 3:
            break
    else:
        times_100 = 0

  0%|     | 0/5000 [00:00<?, ?it/s]     

# TEST


In [11]:
model.eval()
evald = []
evaldDict = {'perf':[],
             'scale':[]}
for nr in [1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40]:
#for nr in range(1,40,):
    perfs = []
    for batch_idx, iv in enumerate(X):
        iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(0).to(device)
        iv = iv.unsqueeze(-1)
        iv = iv.repeat(1,1,1,1,nr)
        iv = iv.reshape(1,1,1,-1)
        tv = Y[batch_idx].to(device)
        out = model(iv)
        loss = loss_func(out[:, -1, :],
                         torch.cuda.LongTensor([tv]))


        perfs.append((torch.argmax(out[:, -1, :], dim=-1) ==
                      tv).sum().item())
    evaldDict['perf'].append(sum(perfs)/len(perfs))
    evaldDict['scale'].append(nr)
    print(nr, sum(perfs)/len(perfs))
    evald.append([nr, sum(perfs)/(len(perfs)*1.0)])
scale_perfs = pd.DataFrame(evaldDict)
directory = "perf"
if not os.path.exists(directory):
    os.makedirs(directory)
scale_perfs.to_pickle(join("perf", "sithcon_braille_test.dill"))

1 0.16279069767441862
2 0.627906976744186
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 1.0
17 1.0
18 1.0
19 1.0
20 1.0
30 1.0
40 1.0
